# Segmenting and Clustering Neighborhoods in Toronto

In [62]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup


In [3]:
import requests
from pandas.io.json import json_normalize 

In [63]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [64]:
soup = BeautifulSoup(data, 'html.parser')

In [89]:
postalCodeList = []
boroughList = []
neighborhoodList = []

In [90]:
soup.find('table').find_all('tr')
soup.find('table').find_all('tr')
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [91]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.strip('\n'))
        boroughList.append(cells[1].text.rstrip('\n'))
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [92]:
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [93]:
toronto_df_drop = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_drop.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [94]:
toronto_df_grouped = toronto_df_drop.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [95]:

for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [96]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,Parkview Hill / Woodbine Gardens
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,Wexford / Maryvale
7,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...


In [97]:
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [98]:
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now merging the data

In [99]:
toronto_df_new = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [100]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\USER\Anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.3                |   py37hc8dfbb8_1         3.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0
  python_abi         conda-forge/win-64::python_abi-3.7-1_cp37m

The following packages will 



==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda




Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\USER\Anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    certifi-2019.9.11          |           py37_0         147 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         860 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/

In [101]:
address = 'Toronto'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Create a map and markers

In [102]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

#### Exploring neighborhoods in Toronto

In [103]:
borough_names = list(toronto_df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [104]:
toronto_df_new = toronto_df_new[toronto_df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_df_new.shape)
toronto_df_new.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188
2,M4L,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [105]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

### Getting necessary data from Foursquare

In [106]:
CLIENT_ID = 'HZ3KXWJBPPSTNXLJDWNC3QD2INGQH25WRAFAOCJTOBH3XKOK' 
CLIENT_SECRET = 'MVXNYVXPFWUUY2DRK1WW3ZBOOW5K0GNWFKKGE5M4VOYBMZ3Z' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HZ3KXWJBPPSTNXLJDWNC3QD2INGQH25WRAFAOCJTOBH3XKOK
CLIENT_SECRET:MVXNYVXPFWUUY2DRK1WW3ZBOOW5K0GNWFKKGE5M4VOYBMZ3Z


In [107]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode'], toronto_df_new['Borough'], 
                                                  toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id=ZYLC4Q3I000O4R32DVJWJJTOTHCGC4O02TXYEPLDAS211SPQ&client_secret=OPKHF1MTRWKRHVR2DAV0IT1IK2H2XZDXJYTCNHVY5L44T55H&v=20180605 \
     &ll=43.653963,-79.387207&radius=500&limit=100".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [108]:
venues_df = pd.DataFrame(venues)


venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1677, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,Japango,43.655268,-79.385165,Sushi Restaurant
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Rolltation,43.654918,-79.387424,Japanese Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Sansotei Ramen 三草亭,43.655157,-79.386501,Ramen Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,Karine's,43.653699,-79.390743,Breakfast Spot


In [109]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()


,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,43,43,43,43,43,43
M4K,East Toronto,The Danforth West / Riverdale,43,43,43,43,43,43
M4L,East Toronto,India Bazaar / The Beaches West,43,43,43,43,43,43
M4M,East Toronto,Studio District,43,43,43,43,43,43
M4N,Central Toronto,Lawrence Park,43,43,43,43,43,43
M4P,Central Toronto,Davisville North,43,43,43,43,43,43
M4R,Central Toronto,North Toronto West,43,43,43,43,43,43
M4S,Central Toronto,Davisville,43,43,43,43,43,43
M4T,Central Toronto,Moore Park / Summerhill East,43,43,43,43,43,43


In [110]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1677, 35)


,PostalCode,Borough,Neighborhoods,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop,...,Plaza,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Smoke Shop,Sushi Restaurant,Tapas Restaurant,University,Vegetarian / Vegan Restaurant
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,M4E,East Toronto,The Beaches,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [111]:

toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 35)


,PostalCode,Borough,Neighborhoods,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop,...,Plaza,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Smoke Shop,Sushi Restaurant,Tapas Restaurant,University,Vegetarian / Vegan Restaurant
0,M4E,East Toronto,The Beaches,0.023256,0.069767,0.023256,0.023256,0.023256,0.046512,0.023256,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
1,M4K,East Toronto,The Danforth West / Riverdale,0.023256,0.069767,0.023256,0.023256,0.023256,0.046512,0.023256,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
2,M4L,East Toronto,India Bazaar / The Beaches West,0.023256,0.069767,0.023256,0.023256,0.023256,0.046512,0.023256,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
3,M4M,East Toronto,Studio District,0.023256,0.069767,0.023256,0.023256,0.023256,0.046512,0.023256,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
4,M4N,Central Toronto,Lawrence Park,0.023256,0.069767,0.023256,0.023256,0.023256,0.046512,0.023256,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
5,M4P,Central Toronto,Davisville North,0.023256,0.069767,0.023256,0.023256,0.023256,0.046512,0.023256,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
6,M4R,Central Toronto,North Toronto West,0.023256,0.069767,0.023256,0.023256,0.023256,0.046512,0.023256,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
7,M4S,Central Toronto,Davisville,0.023256,0.069767,0.023256,0.023256,0.023256,0.046512,0.023256,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
8,M4T,Central Toronto,Moore Park / Summerhill East,0.023256,0.069767,0.023256,0.023256,0.023256,0.046512,0.023256,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
9,M4V,Central Toronto,Summerhill West / Rathnelly / South Hill / For...,0.023256,0.069767,0.023256,0.023256,0.023256,0.046512,0.023256,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256


In [112]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar
1,M4K,East Toronto,The Danforth West / Riverdale,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar
2,M4L,East Toronto,India Bazaar / The Beaches West,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar
3,M4M,East Toronto,Studio District,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar
4,M4N,Central Toronto,Lawrence Park,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar
5,M4P,Central Toronto,Davisville North,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar
6,M4R,Central Toronto,North Toronto West,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar
7,M4S,Central Toronto,Davisville,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar
8,M4T,Central Toronto,Moore Park / Summerhill East,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar
9,M4V,Central Toronto,Summerhill West / Rathnelly / South Hill / For...,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar


### Clustering

In [113]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


C:\Users\USER\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:972: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [114]:
toronto_merged = toronto_df_new.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar
1,M4K,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar
2,M4L,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar


In [115]:
# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar
21,M5L,Downtown Toronto,Commerce Court / Victoria Hotel,43.648198,-79.379817,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar
22,M5N,Central Toronto,Roselawn,43.711695,-79.416936,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar
23,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar
24,M5R,Central Toronto,The Annex / North Midtown / Yorkville,43.672710,-79.405678,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar
25,M5S,Downtown Toronto,University of Toronto / Harbord,43.662696,-79.400049,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar
26,M5T,Downtown Toronto,Kensington Market / Chinatown / Grange Park,43.653206,-79.400049,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar
27,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...,43.628947,-79.394420,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar
20,M5K,Downtown Toronto,Toronto Dominion Centre / Design Exchange,43.647177,-79.381576,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar
28,M5W,Downtown Toronto,Stn A PO Boxes,43.646435,-79.374846,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Hotel,Art Museum,Arts & Crafts Store,Bar


### Visualizing the clusters

In [116]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

We can see that all the neighborhoods fall into one cluster which are the areas with cafe, restaurants, supermarkets etc